In [1]:
from functools import partial
from pathlib import Path
from typing import Optional, Tuple

import cv2
import fire
import numpy as np
import torch
import torch.nn.functional as F
from accelerate import Accelerator
from PIL import Image
from scipy.sparse.linalg import eigsh
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.decomposition import PCA
from torchvision.utils import draw_bounding_boxes
from tqdm import tqdm

import extract_utils as utils

## Extract Eigen Vectors

In [2]:
class args:
    images_root = '/data/home/mukhotij/internship_2022/deep_spectral/data/VOC2012/images'
    features_dir = '/data/home/mukhotij/internship_2022/deep_spectral/data/VOC2012/features/dino_vits16'
    output_dir = '/data/home/mukhotij/internship_2022/deep_spectral/data/VOC2012/eigs/laplacian'
    which_matrix = 'laplacian'
    which_color_matrix = 'knn'
    which_features = 'k' # Again why just k features? Why not q and v?
    normalize = True
    threshold_at_zero = True
    lapnorm = True
    K = 20 # 20 classes for PASCAL VOC?
    image_downsample_factor = None
    image_color_lambda = 1.0 # Set to 0 if dense image features is not required
    multiprocessing = 0

In [3]:
# Create output directory

utils.make_output_dir(args.output_dir)

In [4]:
# Create dictionary of arguments

kwargs = dict(K=args.K,
              which_matrix=args.which_matrix,
              which_features=args.which_features,
              which_color_matrix=args.which_color_matrix,
              normalize=args.normalize,
              threshold_at_zero=args.threshold_at_zero,
              images_root=args.images_root,
              output_dir=args.output_dir,
              image_downsample_factor=args.image_downsample_factor,
              image_color_lambda=args.image_color_lambda,
              lapnorm=args.lapnorm)
print (kwargs)

{'K': 20, 'which_matrix': 'laplacian', 'which_features': 'k', 'which_color_matrix': 'knn', 'normalize': True, 'threshold_at_zero': True, 'images_root': '/data/home/mukhotij/internship_2022/deep_spectral/data/VOC2012/images', 'output_dir': '/data/home/mukhotij/internship_2022/deep_spectral/data/VOC2012/eigs/laplacian', 'image_downsample_factor': None, 'image_color_lambda': 1.0, 'lapnorm': True}


In [5]:
inputs = list(enumerate(sorted(Path(args.features_dir).iterdir())))

In [6]:
# Try out the Eigen vector operation with a single input from the inputs entries

inp = inputs[0]
print (inp)
    

(0, PosixPath('/data/home/mukhotij/internship_2022/deep_spectral/data/VOC2012/features/dino_vits16/2007_000027.pth'))


In [7]:
def compute_eigen(
    inp: Tuple[int, str], 
    K: int, 
    images_root: str,
    output_dir: str,
    which_matrix: str = 'laplacian',
    which_features: str = 'k',
    normalize: bool = True,
    lapnorm: bool = True,
    which_color_matrix: str = 'knn',
    threshold_at_zero: bool = True,
    image_downsample_factor: Optional[int] = None,
    image_color_lambda: float = 10,
):
    index, features_file = inp

    # Load the features file
    data_dict = torch.load(features_file, map_location='cpu')
    image_id = data_dict['id']

    # Load the output file storing the eigen vectors
    output_file = str(Path(args.output_dir) / f'{image_id}.pth')
    output_file_no_img = str(Path(args.output_dir) / f'{image_id}.pth')

    if Path(output_file).is_file():
        print (f'Skipping existing file {str(output_file)}')

    # IMPORTANT: Load affinity matrix - Careful here
    feats = data_dict[args.which_features].squeeze().cuda()
    # print(feats.shape)

    if args.normalize:
        feats = F.normalize(feats, p=2, dim=-1)

    if args.which_matrix in ['matting_laplacian', 'laplacian']:

        # Get sizes
        B, C, H, W, P, H_patch, W_patch, H_pad, W_pad = utils.get_image_sizes(data_dict)
        if args.image_downsample_factor is None:
            image_downsample_factor = P
        H_pad_lr, W_pad_lr = H_pad // image_downsample_factor, W_pad // image_downsample_factor

        # This probably didn't happen
        if (H_patch, W_patch) != (H_pad_lr, W_pad_lr):
            print ('This happened')
            feats = F.interpolate(
                feats.T.reshape(1, -1, H_patch, W_patch), 
                size=(H_pad_lr, W_pad_lr), mode='bilinear', align_corners=False
            ).reshape(-1, H_pad_lr * W_pad_lr).T

        ## Computing feature affinities
        W_feat = (feats @ feats.T)
        if args.threshold_at_zero:
            W_feat = (W_feat * (W_feat > 0))
        W_feat = W_feat / W_feat.max()
        W_feat = W_feat.cpu().numpy()


        ## Color affinities - If we are fusing with color affinities, then load the image and compute
        if args.image_color_lambda > 0:
            # Load image
            image_file = str(Path(args.images_root) / f'{image_id}.jpg')
            image_lr = Image.open(image_file).resize((W_pad_lr, H_pad_lr), Image.BILINEAR)
            image_lr = np.array(image_lr) / 255.
            # print (image_lr.shape)

            # Color affinities
            if args.which_color_matrix == 'knn':
                W_lr = utils.knn_affinity(image_lr / 255)
            elif args.which_color_matrix == 'rw':
                W_lr = utils.knn_affinity(image_lr / 255)

            # Convert to dense numpy array
            W_color = np.array(W_lr.todense().astype(np.float32))

        else:
            W_color = 0

        # Combine
        W_comb = W_feat + W_color * args.image_color_lambda
        D_comb = np.array(utils.get_diagonal(W_comb).todense())
        D_comb_no_img = np.array(utils.get_diagonal(W_feat).todense())

        # Extract eigenvectors
        if args.lapnorm:
            try:
                eigenvalues, eigenvectors = eigsh(D_comb - W_comb, k=args.K, sigma=0, which='LM', M=D_comb)
                eigenvalues_no_img, eigenvectors_no_img = eigsh(D_comb_no_img - W_feat, k=args.K, sigma=0, which='LM', M=D_comb_no_img)
            except:
                eigenvalues, eigenvectors = eigsh(D_comb - W_comb, k=args.K, which='SM', M=D_comb)
                eigenvalues_no_img, eigenvectors_no_img = eigsh(D_comb_no_img - W_feat, k=args.K, which='SM', M=D_comb)
        else:
            try:
                eigenvalues, eigenvectors = eigsh(D_comb - W_comb, k=args.K, sigma=0, which='LM')
                eigenvalues_no_img, eigenvectors_no_img = eigsh(D_comb_no_img - W_feat, k=args.K, sigma=0, which='LM')
            except:
                eigenvalues, eigenvectors = eigsh(D_comb - W_comb, k=args.K, which='SM')
                eigenvalues_no_img, eigenvectors_no_img = eigsh(D_comb_no_img - W_feat, k=args.K, which='SM')

    eigenvalues, eigenvectors = torch.from_numpy(eigenvalues), torch.from_numpy(eigenvectors.T).float()
    eigenvalues_no_img, eigenvectors_no_img = torch.from_numpy(eigenvalues_no_img), torch.from_numpy(eigenvectors_no_img.T).float()

    # Sign ambiguity
    for k in range(eigenvectors.shape[0]):
        if 0.5 < torch.mean((eigenvectors[k] > 0).float()).item() < 1.0:  # reverse segment
            eigenvectors[k] = 0 - eigenvectors[k]
        if 0.5 < torch.mean((eigenvectors_no_img[k] > 0).float()).item() < 1.0:  # reverse segment
            eigenvectors_no_img[k] = 0 - eigenvectors_no_img[k]

    # Save dict
    output_dict = {'eigenvalues': eigenvalues, 'eigenvectors': eigenvectors}
    output_dict_no_img = {'eigenvalues': eigenvalues_no_img, 'eigenvectors': eigenvectors_no_img}

    torch.save(output_dict, output_file)
    torch.save(output_dict_no_img, output_file_no_img)


In [ ]:
fn = partial(compute_eigen, **kwargs)
inputs = list(enumerate(sorted(Path(args.features_dir).iterdir())))
utils.parallel_process(inputs, fn, args.multiprocessing)

  4%|▎         | 628/17125 [05:16<2:55:46,  1.56it/s] 

In [22]:
# Compute Eigen values and Eigen vectors of affinity matrix by different methods

# # Eigenvectors of affinity matrix
# if which_matrix == 'affinity_torch':
#     W = feats @ feats.T
#     if threshold_at_zero:
#         W = (W * (W > 0))
#     eigenvalues, eigenvectors = torch.eig(W, eigenvectors=True)
#     eigenvalues = eigenvalues.cpu()
#     eigenvectors = eigenvectors.cpu()

# # Eigenvectors of affinity matrix with scipy
# elif which_matrix == 'affinity_svd':        
#     USV = torch.linalg.svd(feats, full_matrices=False)
#     eigenvectors = USV[0][:, :K].T.to('cpu', non_blocking=True)
#     eigenvalues = USV[1][:K].to('cpu', non_blocking=True)

# # Eigenvectors of affinity matrix with scipy
# elif which_matrix == 'affinity':
#     W = (feats @ feats.T)
#     if threshold_at_zero:
#         W = (W * (W > 0))
#     W = W.cpu().numpy()
#     eigenvalues, eigenvectors = eigsh(W, which='LM', k=K)
#     eigenvectors = torch.flip(torch.from_numpy(eigenvectors), dims=(-1,)).T


# # We implement the one using the Laplacian matrix but check the others too -- ABLATION!!
# if args.which_matrix in ['matting_laplacian', 'laplacian']:
    
#     # Get sizes
#     B, C, H, W, P, H_patch, W_patch, H_pad, W_pad = utils.get_image_sizes(data_dict)
#     if args.image_downsample_factor is None:
#         image_downsample_factor = P
#     H_pad_lr, W_pad_lr = H_pad // image_downsample_factor, W_pad // image_downsample_factor
    
#     # This probably didn't happen
#     if (H_patch, W_patch) != (H_pad_lr, W_pad_lr):
#         print ('This happened')
#         feats = F.interpolate(
#             feats.T.reshape(1, -1, H_patch, W_patch), 
#             size=(H_pad_lr, W_pad_lr), mode='bilinear', align_corners=False
#         ).reshape(-1, H_pad_lr * W_pad_lr).T
    
#     ## Computing feature affinities
#     W_feat = (feats @ feats.T)
#     if args.threshold_at_zero:
#         W_feat = (W_feat * (W_feat > 0))
#     W_feat = W_feat / W_feat.max()
#     W_feat = W_feat.cpu().numpy()

    
#     ## Color affinities - If we are fusing with color affinities, then load the image and compute
#     if args.image_color_lambda > 0:
#         # Load image
#         image_file = str(Path(args.images_root) / f'{image_id}.jpg')
#         image_lr = Image.open(image_file).resize((W_pad_lr, H_pad_lr), Image.BILINEAR)
#         image_lr = np.array(image_lr) / 255.
#         print (image_lr.shape)
    
#         # Color affinities
#         if args.which_color_matrix == 'knn':
#             W_lr = utils.knn_affinity(image_lr / 255)
#         elif args.which_color_matrix == 'rw':
#             W_lr = utils.knn_affinity(image_lr / 255)
        
#         # Convert to dense numpy array
#         W_color = np.array(W_lr.todense().astype(np.float32))
        
#     else:
#         W_color = 0
        
#     # Combine
#     W_comb = W_feat + W_color * args.image_color_lambda
#     D_comb = np.array(utils.get_diagonal(W_comb).todense())
#     D_comb_no_img = np.array(utils.get_diagonal(W_feat).todense())
    
#     # Extract eigenvectors
#     if args.lapnorm:
#         try:
#             eigenvalues, eigenvectors = eigsh(D_comb - W_comb, k=args.K, sigma=0, which='LM', M=D_comb)
#             eigenvalues_no_img, eigenvectors_no_img = eigsh(D_comb_no_img - W_feat, k=args.K, sigma=0, which='LM', M=D_comb_no_img)
#         except:
#             eigenvalues, eigenvectors = eigsh(D_comb - W_comb, k=args.K, which='SM', M=D_comb)
#             eigenvalues_no_img, eigenvectors_no_img = eigsh(D_comb_no_img - W_feat, k=args.K, which='SM', M=D_comb)
#     else:
#         try:
#             eigenvalues, eigenvectors = eigsh(D_comb - W_comb, k=args.K, sigma=0, which='LM')
#             eigenvalues_no_img, eigenvectors_no_img = eigsh(D_comb_no_img - W_feat, k=args.K, sigma=0, which='LM')
#         except:
#             eigenvalues, eigenvectors = eigsh(D_comb - W_comb, k=args.K, which='SM')
#             eigenvalues_no_img, eigenvectors_no_img = eigsh(D_comb_no_img - W_feat, k=args.K, which='SM')

In [23]:
# eigenvalues, eigenvectors = torch.from_numpy(eigenvalues), torch.from_numpy(eigenvectors.T).float()
# eigenvalues_no_img, eigenvectors_no_img = torch.from_numpy(eigenvalues_no_img), torch.from_numpy(eigenvectors_no_img.T).float()

# # Sign ambiguity
# for k in range(eigenvectors.shape[0]):
#     if 0.5 < torch.mean((eigenvectors[k] > 0).float()).item() < 1.0:  # reverse segment
#         eigenvectors[k] = 0 - eigenvectors[k]
#     if 0.5 < torch.mean((eigenvectors_no_img[k] > 0).float()).item() < 1.0:  # reverse segment
#         eigenvectors_no_img[k] = 0 - eigenvectors_no_img[k]

In [24]:
# # Save dict
# output_dict = {'eigenvalues': eigenvalues, 'eigenvectors': eigenvectors}
# torch.save(output_dict, output_file)


In [ ]:
compute_eigen()